# Data Exploration - Phase II
Shpaner, Leonid  
Robinson, Christopher  
Luis-Estrada, Jose  

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library

/content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library


In [3]:
####################################
## import the requisite libraries ##
####################################
import os
import csv
import pandas as pd
import numpy as np

In [4]:
# check current working directory
current_directory = os.getcwd()
current_directory

'/content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library'

### Assign Paths to Folders

In [5]:
# path to the data file
data_path1 = '/content/drive/Shareddrives/Capstone - Best Group/' \
           + 'Final_Data_20220719/CrimesLA_Streets_with_Crimes.csv'

# path to data folder
data_path2 = '/content/drive/Shareddrives/Capstone - Best Group/navigating_crime/'\
             'Data Folder/'

# path to the image library
image_path = '/content/drive/Shareddrives/Capstone - Best Group/navigating_crime'\
          + '/Image Folder'        

In [8]:
# read in the training .csv file to a dataframe using pandas
df_ml = pd.read_csv(data_path2 + 'train_set.csv', low_memory=False)
df_ml.head()

,OBJECTID,Zip_Code,Accuracy,Crime_Cost,StreetOID,DR_NO,TIME_OCC,AREA,Rpt_Dist_No,Part_1_2,Vict_Age,Premis_Cd,Weapon_Used_Cd,Days_Disclosure,Street_Type,Gender,Crime_Code
0,69772,90024.0,50.0,2.0,27815,210814900.0,750.0,8.0,833.0,2.0,40.0,102.0,400.0,0,7,1,0
1,131641,90037.0,50.0,3.0,51117,210313311.0,1100.0,3.0,397.0,1.0,32.0,101.0,106.0,0,8,2,0
2,143866,90016.0,50.0,1.0,55526,200705585.0,1420.0,7.0,785.0,1.0,46.0,101.0,102.0,0,7,2,1
3,189323,90003.0,50.0,0.0,72325,221213641.0,910.0,12.0,1268.0,2.0,25.0,102.0,400.0,0,8,1,0
4,89956,91331.0,50.0,2.0,35427,221606939.0,1625.0,16.0,1612.0,2.0,63.0,108.0,400.0,0,6,2,0


In [9]:
# show the columns of the dataframe for inspection
df_ml.columns

Index(['OBJECTID', 'Zip_Code', 'Accuracy', 'Crime_Cost', 'StreetOID', 'DR_NO',
       'TIME_OCC', 'AREA', 'Rpt_Dist_No', 'Part_1_2', 'Vict_Age', 'Premis_Cd',
       'Weapon_Used_Cd', 'Days_Disclosure', 'Street_Type', 'Gender',
       'Crime_Code'],
      dtype='object')

In [10]:
# re-inspect the shape of the dataframe. This is also done on EDA file.
df_ml.shape

(128205, 17)